# About this Notebook

Data Wrangling can be defined as the process to:
* gather
* assess
* clean
data in support of data analytics tasks.

This notebook contains the code to extract data related to Master Swim Performances and save it in a local format that supports Data Exploration activities.

## Supported Data Sources

Currently data is only extracted from the Canadian swimming site that collects the results for Masters Swimmers.

In [1]:
import requests
import lxml.html as lh
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

# Canadian Master Swimming Results Collection 

Starting with the information I know how to access from personal experience.

## Gather - www.swimming.ca
The following code will scrape the available data from the www.swimming.ca website for Masters Swimmers. The code is based on the website structure that exists during the time period Aug - Oct 2020.

The website has a layered approach to finding results.
1. Navigate to the page that has a list of meet-results
2. Filter the list of meet results by **Season**, **Province**, **Month**
3. Select the meet of interest in the filtered list
4. At this point it is possible to see all the results for each club that participated, or the results by event.  I will be collecting all the results by participating club.

I did contact Swim Canada to see if there was an API that would allow me to use an approach other than web scraping to get the results, but so far they have not been able to support my request.

For the parameters **Seasion**, **Province** and **Month**, there are few points to note:
* Season - the swim season in Canada is split over 2 calendar years and runs Sep to Aug.  So the swim season 2019/2020 will be the period 1 Sep 2019 to 31 Aug 2020.  The value passed to the **Season** parameter is the last 2 numbers in the second part of the season identifier.  So to access swim meets from Nov 2019, the season parameter will be 20 ('season'='20')
* Month - the month must be specified as a number from 1 to 12.  It is not possible to have all the months in a single season being available at one time
* Province - It is possible to filter by province/territory or use all provinces. To access an individual province/territory, the provinces/territories are numbered from 1 to 11 in alphabetical order.  So to get only Alberta, 'province'='1' and to get Yukon, 'province'='11'.  The supported Territories are Northwest Territories and Yukon.  For all provinces, no number is provided to province (province = '')

Since I am only interested in Masters results, when looking at the meet list, I will look for MEET TYPE  of Masters.  This will miss some swimmers that compete in non-Masters meets as an Open Master.  I plan to come back and find these results after collecting the Masters Meets

The only URL specified in my code is the URL associated with step 1 above, navigating to the meet-results page.  The subsequent URLs will be recovered from the information available when the various filters are applied.

In [2]:
# URLs of interest
url_meetlist ='https://www.swimming.ca/en/events-results/meet-results/'

### Explore Structure of html pages
In order to recover the information of interest, the way the information is presented on the various html pages needs to be understood.

Once I know how to capture the data of interest, I will create functions to iterate through diffferent seasons, months and meets.

The information I am interested in can be grouped into 3 categories.

**Swimmer Information**
* sw_id_num - unique swimmer id number
* sw_name - swimmers name
* sw_yob - swimmers year of birth

Note that swimmers age is determined by the age on Dec 31st of the year in which the competition happened.

**Meet Information**

* mt_id_num - unique meet id number
* mt_name - meet name
* mt_date - first day of meet
* mt_duration - number of days the swim meet lasted
* mt_pool_name - name of the pool
* mt_pool_address - address of the pool
* mt_sc_lc - indication if meet was a long course or short course meet

**Race Information**
* rc_dist - distance of race - eg 25m, 100m, 400m
* rc_stroke - stroke of race - freestyle, backstroke, butterfly, breaststroke, IM
* rc_time - time it took the swimmer to complete the distance
* rc_round - heat, final, split of longer swim or part of relay
* sw_id_num - unique swimmer id
* mt_id_num - unique meet id

I am also confident that at some point I will want club information, but at this point I'm not sure what I would do with it.

I also have not decided the best way to store the information I scrape.  Initially, I will put it in 3 different csv's, but would like to explore a graph database.  I have not used them before and would like to learn.

In [3]:
# start with the 2019/2020 season, noting that the season came to an abrupt end in March 2020 because of Covid19
# using November since I know there is a masters meet in Nov 2019
response=requests.get('https://www.swimming.ca/en/events-results/meet-results/', params={'season':'20', 'province':'', 'month':'11'})
#sanity check that my parameter specification worked as expected
response.url

'https://www.swimming.ca/en/events-results/meet-results/?season=20&province=&month=11'

In [4]:
#use Beautiful Soup to parse the returned page
meetList_resp = BeautifulSoup(response.text, 'lxml')

In [5]:
#find the masters swim meets in the response text
meetList = []
for item in meetList_resp.find_all('tr'):
    if item.contents[5].contents[0] == "Masters":
        temp_dict = {}
        temp_dict['meet_date'] = item.contents[0].contents[0].contents[0]
        temp_dict['meet_url'] = item.contents[1].a.attrs['href']
        temp_dict['meet_prov'] = item.contents[2].contents[0]
        temp_dict['meet_host'] = item.contents[3].contents[0].contents[0]
        temp_dict['meet_course'] = item.contents[4].contents[0]
        temp_dict['meet_type'] = item.contents[5].contents[0]
        meetList.append(temp_dict)
        ##used for debug/development
        #print(item.contents[0].contents[0].contents[0])
        #print(item.contents[1].a.attrs['href'])
        #print(item.contents[2].contents[0])
        #print(item.contents[3].contents[0].contents[0])
        #print(item.contents[4].contents[0])
        #print(item.contents[5].contents[0])
print("number of masters meets", len(meetList))

number of masters meets 8


So 8 masters meets were held in Canada in Nov 2019.

Now use the meet_url to get the results for that meet.  The use Beautiful Soup to parse the returned page

In [6]:
r2_url = meetList[0]['meet_url']
r2 = requests.get(r2_url)
temp_r2 = BeautifulSoup(r2.text, 'lxml')

My next step was to create a list of clubs that participated. I looked through the html on the page and determined the information I was interested in was held in 'option' tags.

In [23]:
clubList = []
for item in temp_r2.find_all('option'):
    #print(item.contents[0])
    temp_dict = {}
    if "Events" in item.contents[0]:
        #print(item)
        break
    elif "Participants" not in item.contents[0]:
        temp_dict['club_res_url'] = item.attrs['data-href']
        temp_dict['club_number'] = item.attrs['value']
        temp_dict['club_name'] = item.contents[0]
        clubList.append(temp_dict)
        ## used for dev/debug
        #print(item.attrs['data-href'])
        #print(item.attrs['value'])
        #print(item.contents[0])
        
print("number of clubs in the meet", len(clubList))
print("information about each club") 
clubList[0] 

number of clubs in the meet 11
information about each club


{'club_res_url': 'https://www.swimming.ca/en/meet/619320/?faction=73928',
 'club_number': '73928',
 'club_name': 'Edmonton Masters Swim Club'}

The next step is to get the club results and retrieve the information about each swimmer

In [8]:
r3_url = clubList[0]['club_res_url']
r3 = requests.get(r3_url)
temp_r3 = BeautifulSoup(r3.text, 'lxml')

After examining the HTML for the per club results page, the results of interest seem to be in the first 4 tables:
* table 0 - men's results
* table 1 - women's results
* table 2 - relay results
* table 3 - meet resource info

In [9]:
temp_table = temp_r3.find_all("table")
len(temp_table)

5

In [72]:
#look at the results for the men's table
count = 0
swimmer_list = []
race_list = []
for item in temp_table[0].find_all('tr'):
    temp_sw_dict = {}
    temp_rc_dict = {}
    if item.has_attr('class'):
        print("new swimmer!!")
        #print("row count", count, "length of info", len(item), "contents", item)
        #print("swimmer url", item.find('a').attrs['href'])
        #print("swimmer id", item.find('a').attrs['href'].split('/')[5])
        #print("swimmer name", item.find('a').contents[0])
        #print("swimmer YOB", item.find('th').contents[1][3:7])
        # need swim id for both the swimmer and the race dictionary
        sw_id = item.find('a').attrs['href'].split('/')[5]
        sw_yob = item.find('th').contents[1][3:7]
        temp_sw_dict['sw_url'] = item.find('a').attrs['href']
        temp_sw_dict['sw_id'] = sw_id
        temp_sw_dict['sw_name'] = item.find('a').contents[0]
        temp_sw_dict['sw_yob'] = sw_yob
        swimmer_list.append(temp_sw_dict)
    #print(count)
    #if item.children[0].has_attr('class'):
    #    print("race time and dist available")
    #    print(item.children[0])
    elif count != 0:
        #print("row count", count, len(item.contents))
        #for subItem in item:
        #    print(len(subItem), subItem.string)
        #print("gender", item.contents[0].find('a').attrs['data-query-gender'])
        #print("race distance and stroke", item.contents[0])
        #print("race distance and stroke", item.contents[0].find('span').find('a').contents[0])
        #print("race round", item.contents[1].contents[0])
        #print("race time", item.contents[3].contents[0])
        #get the gender for both swim and race dict
        sw_gender = item.contents[0].find('a').attrs['data-query-gender']
        temp_rc_dict['sw_id'] = sw_id
        temp_rc_dict['sw_yob'] = sw_yob
        temp_rc_dict['sw_gender'] = sw_gender
        #temp_sw_dict['sw_gender'] = sw_gender
        temp_rc_dict['rc_dist_stroke'] = item.contents[0].find('span').find('a').contents[0]
        temp_rc_dict['rc_round'] = item.contents[1].contents[0]
        temp_rc_dict['rc_time'] = item.contents[3].contents[0]
        race_list.append(temp_rc_dict)
    else:
        print("row count", count)
    
    #append the temp dict to the lists
    #swimmer_list.append(temp_sw_dict)
    #race_list.append(temp_rc_dict)
    count += 1
    # just print out some to see the pattern
    #if count == 11:
    #    break

row count 0
new swimmer!!
new swimmer!!
new swimmer!!
new swimmer!!
new swimmer!!
new swimmer!!
new swimmer!!
new swimmer!!


In [73]:
swimmer_list

[{'sw_url': 'https://www.swimming.ca/en/swimmer/4764413/',
  'sw_id': '4764413',
  'sw_name': 'Dal Bello, George',
  'sw_yob': '1990'},
 {'sw_url': 'https://www.swimming.ca/en/swimmer/5396447/',
  'sw_id': '5396447',
  'sw_name': 'Edelenbosch, Florian',
  'sw_yob': '1992'},
 {'sw_url': 'https://www.swimming.ca/en/swimmer/4040463/',
  'sw_id': '4040463',
  'sw_name': 'Gordon, Graeme',
  'sw_yob': '1979'},
 {'sw_url': 'https://www.swimming.ca/en/swimmer/4456009/',
  'sw_id': '4456009',
  'sw_name': 'Lewis, Barry',
  'sw_yob': '1962'},
 {'sw_url': 'https://www.swimming.ca/en/swimmer/4568512/',
  'sw_id': '4568512',
  'sw_name': 'Morrison, Zachary',
  'sw_yob': '1995'},
 {'sw_url': 'https://www.swimming.ca/en/swimmer/4469236/',
  'sw_id': '4469236',
  'sw_name': 'Parrish, Adam',
  'sw_yob': '1962'},
 {'sw_url': 'https://www.swimming.ca/en/swimmer/4941482/',
  'sw_id': '4941482',
  'sw_name': 'Smits, Nolan',
  'sw_yob': '1985'},
 {'sw_url': 'https://www.swimming.ca/en/swimmer/4568665/',
  '

In [74]:
race_list

[{'sw_id': '4764413',
  'sw_yob': '1990',
  'sw_gender': 'M',
  'rc_dist_stroke': '25m Freestyle',
  'rc_round': 'Final',
  'rc_time': '12.78'},
 {'sw_id': '4764413',
  'sw_yob': '1990',
  'sw_gender': 'M',
  'rc_dist_stroke': '50m Freestyle',
  'rc_round': 'Final',
  'rc_time': '26.97'},
 {'sw_id': '4764413',
  'sw_yob': '1990',
  'sw_gender': 'M',
  'rc_dist_stroke': '100m Freestyle',
  'rc_round': 'Final',
  'rc_time': '1:03.05'},
 {'sw_id': '4764413',
  'sw_yob': '1990',
  'sw_gender': 'M',
  'rc_dist_stroke': '50m Butterfly',
  'rc_round': 'Final',
  'rc_time': '30.30'},
 {'sw_id': '4764413',
  'sw_yob': '1990',
  'sw_gender': 'M',
  'rc_dist_stroke': '50m Freestyle Laps',
  'rc_round': '4 × 50m Medley, Final',
  'rc_time': '27.75'},
 {'sw_id': '4764413',
  'sw_yob': '1990',
  'sw_gender': 'M',
  'rc_dist_stroke': '50m Freestyle Laps',
  'rc_round': '4 × 50m Freestyle, Final',
  'rc_time': '27.75'},
 {'sw_id': '4764413',
  'sw_yob': '1990',
  'sw_gender': 'X',
  'rc_dist_stroke': 

## Assess - www.swimming.ca

## Clean - www.swimming.ca